In [1]:
from pyspark.sql import SparkSession
from vars import *
from datetime import date
from functions import loadConfigs
import os
from pyspark.sql.functions import explode
from pyspark.sql.functions import lit
from pyspark.sql import functions as F
from delta import *

# minio credentials
minio_endpoint = "http://minio:9000"
minio_access_key = "iQekrU3nA33aKktu2GMo"
minio_secret_key = "1OGQ3m1lWEZVQ31xKfPCKmZVFVb2lXwCorjZr1d1"
minio_region = "us-east-1"
minio_bucket = "reddit-datalake"

builder = loadConfigs(SparkSession.builder)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [13]:
today = date.today().strftime('%Y%m%d')

df = spark.read.format("delta").load(f"s3a://{minio_bucket}/bronze/popular/popular_{today}")

In [14]:
df.show()

+--------------------+-------------------+---------------+-----------+--------+--------------------+--------------+-----------------------------+----------------------+---------------------+------------------------+--------------------+-----------------------+-----------------+---------------+-----------------+--------------------+--------------+--------+-------------+---------+--------+------------+--------+-------+------------------+------------+-------------+-------------+---------------+-------------+--------------------+-----+-------------+--------------------+------+------------------+------+----------+-------+----------------------+----------------+----------+-------+-------------------+----------------------+------------------+-------+--------+-----+---------------------------+--------------------+--------------------+----------------------+--------------------+---------------------+---------------+------+--------------------+--------------------+----------+--------+-----------

In [15]:
from columns import gildings

df_gildings = df.select("id", "author", "gilded", "gildings", "is_video", "score", "subreddit_id", "subreddit", "total_awards_received") \
                      .withColumnRenamed("id", "post_id")

df_gildings = df_gildings.select("*", "gildings.*")

if "gid_1" not in df_gildings.columns:
    df_gildings = df_gildings.withColumn("gid_1", lit(None).cast(IntegerType()))

df_renamed = df_gildings.withColumnRenamed("gid_1", "gild_silver") \
                        .withColumnRenamed("gid_2", "gild_gold") \
                        .withColumnRenamed("gid_3", "gild_platinum")

df_final = df_renamed.dropDuplicates()

df_final = df_final.withColumn("dateid", lit(today))

df_final = df_final.select(gildings)

In [25]:
df_final.orderBy("gilded", ascending=False).show()

+-------+--------------------+------+------------------+-----------+---------+-------------+--------+------+------------+--------------------+--------+
|post_id|              author|gilded|          gildings|gild_silver|gild_gold|gild_platinum|is_video| score|subreddit_id|           subreddit|  dateid|
+-------+--------------------+------+------------------+-----------+---------+-------------+--------+------+------------+--------------------+--------+
|148hozn|           timmeh519|     5|         {1, 5, 2}|          1|        5|            2|   false|125457|    t5_2qh5i|               happy|20230614|
|148ks6u|     SpicyThunder335|     5|         {2, 5, 5}|          2|        5|            5|   false| 13730|   t5_4yrnne|            ModCoord|20230614|
|148wo64|          Riftmarked|     2|      {null, 1, 2}|       null|        1|            2|   false|  1746|    t5_3g1qk|       goldenknights|20230614|
|148rfyk|    Eastern-Truck433|     1|   {null, 1, null}|       null|        1|         n

In [18]:
from pyspark.sql.functions import current_date, date_format

author_df = df_final.groupBy("author","subreddit_id","subreddit").agg(
    F.sum("gild_silver").alias("total_gild_silver"),
    F.sum("gild_gold").alias("total_gild_gold"),
    F.sum("gild_platinum").alias("total_gild_platinum"),
    F.sum("score").alias("total_score")
)

author_df = author_df.withColumn(
    'total_gildings',
    F.col('total_gild_silver') + F.col('total_gild_gold') + F.col('total_gild_platinum')
).orderBy('total_gildings',ascending=False)

author_df = author_df.withColumn("updated_at", date_format(current_date(), "yyyyMMdd"))

In [19]:
author_df.show()

+--------------------+------------+--------------------+-----------------+---------------+-------------------+-----------+--------------+----------+
|              author|subreddit_id|           subreddit|total_gild_silver|total_gild_gold|total_gild_platinum|total_score|total_gildings|updated_at|
+--------------------+------------+--------------------+-----------------+---------------+-------------------+-----------+--------------+----------+
|     SpicyThunder335|   t5_4yrnne|            ModCoord|                2|              5|                  5|      13730|            12|  20230614|
|           timmeh519|    t5_2qh5i|               happy|                1|              5|                  2|     125457|             8|  20230614|
|EminemsDaughterSucks|   t5_2iku5r|    okbuddychicanery|             null|           null|               null|       5087|          null|  20230614|
|           gravityVT|    t5_2td2i|       nbacirclejerk|             null|           null|               n

In [20]:
columns_to_fill = ["total_gild_silver", "total_gild_gold", "total_gild_platinum", "total_score", "total_gildings"]
author_df = author_df.na.fill(0, subsett=columns_to_fill)

In [21]:
author_df.show()

+--------------------+------------+--------------------+-----------------+---------------+-------------------+-----------+--------------+----------+
|              author|subreddit_id|           subreddit|total_gild_silver|total_gild_gold|total_gild_platinum|total_score|total_gildings|updated_at|
+--------------------+------------+--------------------+-----------------+---------------+-------------------+-----------+--------------+----------+
|     SpicyThunder335|   t5_4yrnne|            ModCoord|                2|              5|                  5|      13730|            12|  20230614|
|           timmeh519|    t5_2qh5i|               happy|                1|              5|                  2|     125457|             8|  20230614|
|EminemsDaughterSucks|   t5_2iku5r|    okbuddychicanery|                0|              0|                  0|       5087|             0|  20230614|
|           gravityVT|    t5_2td2i|       nbacirclejerk|                0|              0|                